# Analyze a subset of data

Big data analysis always start with a manageable subset of the data, this allows you to:

* Download the dataset locally if needed,
* Explore it in depth, and
* Experiment with the computations you wish to do.

Once you have your computations ready, you can focus on scaling up!



## Introduce dataset: Airline on-time performance data

In this tutorial, we will analyze **the ["airline on-time performance" dataset](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJhttps://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ) -- a collection of flight records maintained by the U.S. Department of Transportation's Bureau of Transportation Statistics (BTS)**.

This dataset provides information about the on-time performance of domestic flights operated by large air carriers in the United States, including flight delays, cancellations, and diversions. It covers flights operated by 23 major airlines and the records from 1987-present day.

We will work with data from 2003-2022, which is ~70 GB is size on disk.


## Read a subset with pandas

Read data for 1 year, 2022.

In [1]:
import json
import gcsfs

import pandas as pd

We'll look at cloud storage in the next notebook.

In [2]:
token = json.load(open("scripts/credentials.json"))
fs = gcsfs.GCSFileSystem(token=token)
storage_options={"token": token}

In [3]:
files = [f"gcs://{f}" for f in fs.glob("quansight-datasets/airline-ontime-performance/csv/*2022.csv")]

In [4]:
with open('scripts/dtypes.json', 'r') as f:
    dtypes = json.load(f)

In [5]:
# this cell will take ~2.5 minutes to execute on a small/medium machine profile

df_list = []

for file in files:
    df_temp = pd.read_csv(file, dtype=dtypes, storage_options=storage_options)
    df_list.append(df_temp)

In [6]:
df = pd.concat(df_list)

As the above cells execute, [learn more about the dataset](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ).

## Explore the dataset

In [7]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
0,2022,2,4,1,5,4/1/2022 12:00:00 AM,9E,20363,9E,N132EV,...,<NA>,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>
1,2022,2,4,1,5,4/1/2022 12:00:00 AM,9E,20363,9E,N133EV,...,<NA>,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>
2,2022,2,4,1,5,4/1/2022 12:00:00 AM,9E,20363,9E,N133EV,...,<NA>,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>
3,2022,2,4,1,5,4/1/2022 12:00:00 AM,9E,20363,9E,N133EV,...,<NA>,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>
4,2022,2,4,1,5,4/1/2022 12:00:00 AM,9E,20363,9E,N133EV,...,<NA>,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>


In [1]:
# df.describe() -- kernel restarts even with a medium profile

In [9]:
df.columns[:60]

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID',
       'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15',
       'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE',
       'DISTANCE_GROU

### Maximum and average delay in departure?

In [14]:
max_dep_delay = df["DEP_DELAY"].max()

print(f"The maximum departure delay in departure is {max_dep_delay} minutes, or ~{max_dep_delay // 60} hours.")

The maximum departure delay in departure is 3433.0 minutes, or ~57.0 hours.


In [15]:
mean_dep_delay = df["DEP_DELAY"].mean()

print(f"The average departure delay in departure is {round(mean_dep_delay, 2)} minutes.")

The average departure delay in departure is 12.09 minutes.


### Which airport/airline has the most flight departure and arrival delays?

**Airport:**

In [16]:
df.groupby("ORIGIN")["DEP_DELAY"].count().idxmax()

'ATL'

In [17]:
df.groupby("ORIGIN")["ARR_DELAY"].count().idxmax()

'ATL'

That's the code for Hartsfield-Jackson Atlanta International Airport, interesting. :)

**Airline:**

In [18]:
df.groupby("OP_CARRIER")["DEP_DELAY"].count().idxmax()

'WN'

In [19]:
df.groupby("OP_CARRIER")["ARR_DELAY"].count().idxmax()

'WN'

'WN' is the code for Southwest Airlines.

### Your turn: Busiest day/month?

In [20]:
# Your code here.

### Your turn: Total flight cancellations?

In [21]:
# Your code here.

### Total time people spent on a flight in 2022?

In [22]:
time_in_flight = df["ACTUAL_ELAPSED_TIME"].sum()

print(f"People spent a total of {time_in_flight} minutes on a domestic flight, in 2022 in the USA; \nwhich is ~{round(time_in_flight / (60*24*30*12), 2)} years in aggregate.")

People spent a total of 823399905.0 minutes on a domestic flight, in 2022 in the USA; 
which is ~1588.35 years in aggregate.


---

## Next

Let's chat briefly about [storage formats](02-storage-formats.ipynb)!